# Drug Overdose Modeling

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import patsy 
import math

pd.set_option('display.max_columns', 100)
%matplotlib inline

## Import the feature DataFrame

In [2]:
# Read the DF created from the feature_selection notebook
counties = pd.read_pickle('counties_df.pkl')

In [3]:
# Make the column names standardized for use in patsy formulas
counties.columns = [x.replace(' ', '_').replace('-', '_').replace('%', 'Percent') for x in list(counties.columns)]

## Train a model on the appropriate features

In [18]:
formula = "Drug_Overdose_Mortality_Rate ~ 1 + Percent_Unemployed + Household_Income + Graduation_Rate + Percent_Rural + HIV_Prevalence_Rate + Percent_Frequent_Mental_Distress + Percent_Excessive_Drinking"
y, X = patsy.dmatrices(formula, counties, return_type='matrix') 
model = sm.OLS(y, X)

In [19]:
results = model.fit()

In [20]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Drug_Overdose_Mortality_Rate   R-squared:                       0.169
Model:                                      OLS   Adj. R-squared:                  0.166
Method:                           Least Squares   F-statistic:                     47.91
Date:                          Sun, 09 Dec 2018   Prob (F-statistic):           3.67e-62
Time:                                  18:33:38   Log-Likelihood:                -6081.7
No. Observations:                          1655   AIC:                         1.218e+04
Df Residuals:                              1647   BIC:                         1.222e+04
Df Model:                                     7                                         
Covariance Type:                      nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                          -19.2633      6.156     -3.129      0.002     -31.337      -7.189
Percent_Unemployed                   0.8768      0.174      5.036      0.000       0.535       1.218
Household_Income                     0.0001    2.9e-05      3.467      0.001    4.37e-05       0.000
Graduation_Rate                      0.0669      0.035      1.924      0.055      -0.001       0.135
Percent_Rural                        0.0172      0.011      1.597      0.110      -0.004       0.038
HIV_Prevalence_Rate                 -0.0027      0.001     -2.109      0.035      -0.005      -0.000
Percent_Frequent_Mental_Distress     2.2124      0.274      8.084      0.000       1.676       2.749
Percent_Excessive_Drinking          -0.2322      0.099     -2.347      0.019      -0.426      -0.038
==============================================================================
Omnibus:                      535.045   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2169.778
Skew:                           1.518   Prob(JB):                         0.00
Kurtosis:                       7.717   Cond. No.                     1.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression
X = counties.copy().drop(columns=['Drug_Overdose_Mortality_Rate'])
X = X.select_dtypes(exclude='object')
y = counties['Drug_Overdose_Mortality_Rate']
kbest = SelectKBest(f_regression, k=20).fit(X, y)

In [16]:
new_features = []
mask = kbest.get_support()

for boolean, feature in zip(mask, list(counties.columns.values)):
    if boolean:
        new_features.append(feature)

In [17]:
new_features

['State',
 'Age_Adjusted_Mortality',
 'Age_Adjusted_Mortality_(Black)',
 'Child_Mortality_Rate',
 'Child_Mortality_Rate_(Black)',
 'Child_Mortality_Rate_(White)',
 'Infant_Mortality_Rate',
 'Percent_Frequent_Physical_Distress',
 'Percent_Uninsured.1',
 'Segregation_index',
 'Percent_Not_Proficient_in_English',
 'State_ranked',
 'Years_of_Potential_Life_Lost_Rate',
 'Years_of_Potential_Life_Lost_Rate_(Black)',
 'Physically_Unhealthy_Days',
 'Mentally_Unhealthy_Days',
 'Percent_Smokers',
 'Teen_Birth_Rate_(White)',
 'Percent_Some_College',
 '20th_Percentile_Income']